# Imports and utils

In [3]:
from utils import *
import numpy as np
import pickle
import wandb

In [4]:
index_to_english_alphabet=pickle.load(open('vocab_tools/index_to_english_alphabet.pickle', 'rb'))
index_to_hindi_alphabet=pickle.load(open('vocab_tools/index_to_hindi_alphabet.pickle', 'rb'))

In [5]:
hindi_alphabet_to_index=pickle.load(open('vocab_tools/hindi_alphabet_to_index.pickle', 'rb')) 
english_alphabet_to_index=pickle.load(open('vocab_tools/english_alphabet_to_index.pickle', 'rb')) 

In [6]:
X_train=np.load('simple_data/X_train.npy')
X_valid=np.load('simple_data/X_val.npy')

y_train=np.load('simple_data/y_train.npy')
y_valid=np.load('simple_data/y_val.npy')

In [7]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

/opt/anaconda/envs/shared_conda_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataloaders

In [8]:
class Eng_Hind_Dataset(Dataset):

    def __init__(self, in_file, out_file, root_dir='simple_data',device='cuda'):

        self.input = torch.tensor(np.load(root_dir+'/'+in_file))
        self.output = torch.tensor(np.load(root_dir+'/'+out_file))
        
        assert(len(self.input)==len(self.output),"Error: I/O Lengths must be same")
        

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        X=self.input[idx]
        X=X.to(device)
        y=self.output[idx]
        y=y.to(device)
        


        sample = {'input': X, 'output': y}

        return sample

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
device

device(type='cuda')

In [11]:
training_data=Eng_Hind_Dataset("X_train.npy","y_train.npy",device=device)
val_data=Eng_Hind_Dataset("X_val.npy","y_val.npy",device=device)
test_data=Eng_Hind_Dataset("X_test.npy","y_test.npy",device=device)

In [12]:
train_val_sets = torch.utils.data.ConcatDataset([training_data, val_data])

In [13]:
train_val_dataloader = DataLoader(train_val_sets, batch_size=16,shuffle=True)

In [14]:
test_dataloader = DataLoader(test_data, batch_size=16,shuffle=True)

In [15]:
import torch.nn as nn

In [16]:
def cell_type(mode:str='rnn'):
    mode=mode.lower()
    if mode == 'rnn':
        return nn.RNN
    elif mode =='gru':
        return nn.GRU
    else:
        return nn.LSTM
        

# Model

In [13]:
class Encoder(nn.Module):
    """
    Encoder module of the Seq2Seq model.

    Parameters
    ----------
    input_size : int
        Input size, should equal to the source vocabulary size.

    embed_size : int
        Embedding layer's dimension.

    enc_hid_size : int
        Encoder's hidden state size.

    dec_hid_size : int
        Decoder's hidden state size.

    num_layers : int
        Number of encoder layers.

    cell_mode : str
        Type of cell: LSTM, GRU, or RNN.

    dropout : float
        Dropout for the encoder layer.

    is_bi : bool
        Flag indicating if the encoder is bidirectional.

    Attributes
    ----------
    embedding : nn.Embedding
        Embedding layer.

    cell : nn.Module
        LSTM/GRU/RNN cell.

    cell_mode : str
        Type of cell: LSTM, GRU, or RNN.

    is_bi : bool
        Flag indicating if the encoder is bidirectional.

    fc : nn.Linear
        Linear layer to transform the encoder's hidden state.

    Methods
    -------
    forward(input_batch: torch.LongTensor)
        Forward pass of the encoder.

    """
    def __init__(self, input_size, embed_size, enc_hid_size, dec_hid_size, num_layers, cell_mode, dropout, is_bi):
        super().__init__()

        self.embedding = nn.Embedding(input_size, embed_size, padding_idx=english_alphabet_to_index['.'])

        cell = cell_type(cell_mode)

        self.cell = cell(embed_size, enc_hid_size, num_layers, dropout=dropout, bidirectional=is_bi, batch_first=True)
        self.cell_mode = cell_mode
        self.is_bi = is_bi
        if is_bi:
            self.fc = nn.Linear(enc_hid_size * 2, dec_hid_size)
        else:
            self.fc = nn.Linear(enc_hid_size, dec_hid_size)

    def forward(self, input_batch: torch.LongTensor):
        """
        Forward pass of the encoder.

        Parameters
        ----------
        input_batch : torch.LongTensor
            Batched tokenized source sentence of shape [sent len, batch size].

        Returns
        -------
        outputs : torch.Tensor
            Outputs of the LSTM layer.

        hidden : torch.Tensor
            Hidden state of the LSTM layer.

        cell : torch.Tensor (only for LSTM)
            Cell state of the LSTM layer.
        """
        embedded = self.embedding(input_batch)

        if self.cell_mode.lower() == 'lstm':
            outputs, (hidden, cell) = self.cell(embedded)

            if self.is_bi:
                concated = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
                cellconcat = torch.cat((cell[-2, :, :], cell[-1, :, :]), dim=1)
            else:
                concated = hidden[-1, :, :]
                cellconcat = cell[-1, :, :]

            hidden = torch.tanh(self.fc(concated))
            cell = torch.tanh(self.fc(cellconcat))

            return outputs, hidden, cell

        else:
            outputs, hidden = self.cell(embedded)
            if self.is_bi:
                concated = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
            else:
                concated = hidden[-1, :, :]

            hidden = torch.tanh(self.fc(concated))

            return outputs, hidden


In [14]:
class Attention(nn.Module):
    """
    Attention module of the Seq2Seq model.

    Parameters
    ----------
    enc_hid_dim : int
        Encoder's hidden state size.

    dec_hid_dim : int
        Decoder's hidden state size.

    is_bi : bool
        Flag indicating if the encoder is bidirectional.

    Attributes
    ----------
    enc_hid_dim : int
        Encoder's hidden state size.

    dec_hid_dim : int
        Decoder's hidden state size.

    fc1 : nn.Linear
        Linear layer to transform the concatenated input.

    fc2 : nn.Linear
        Linear layer to transform the attention energy.

    Methods
    -------
    forward(encoder_outputs, hidden)
        Forward pass of the attention module.

    """
    def __init__(self, enc_hid_dim, dec_hid_dim, is_bi):
        super().__init__()
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        if is_bi:
            self.fc1 = nn.Linear(enc_hid_dim * 2 + dec_hid_dim, dec_hid_dim)
        else:
            self.fc1 = nn.Linear(enc_hid_dim + dec_hid_dim, dec_hid_dim)

        self.fc2 = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, encoder_outputs, hidden):
        """
        Forward pass of the attention module.

        Parameters
        ----------
        encoder_outputs : torch.Tensor
            Outputs of the encoder. Shape [batch size, src len, enc hid dim * num directions].

        hidden : torch.Tensor
            Hidden state of the decoder. Shape [batch size, dec hid dim].

        Returns
        -------
        attention_weight : torch.Tensor
            Attention weights. Shape [batch size, src len].

        """
        src_len = encoder_outputs.shape[1]
        batch_size = encoder_outputs.shape[0]

        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        outputs = encoder_outputs

        concat = torch.cat((hidden, outputs), dim=2)
        energy = torch.tanh(self.fc1(concat))

        attention = self.fc2(energy).squeeze(dim=2)
        attention_weight = torch.softmax(attention, dim=1)
        return attention_weight


In [15]:
class Decoder(nn.Module):
    """
    Decoder module of the Seq2Seq model.

    Parameters
    ----------
    output_size : int
        Size of the target vocabulary.

    embed_size : int
        Embedding layer's dimension.

    enc_hid_dim : int
        Encoder's hidden state size.

    dec_hid_dim : int
        Decoder's hidden state size.

    num_layers : int
        Number of layers in the decoder.

    cell_mode : str
        Cell type for the decoder (e.g., LSTM, GRU, RNN).

    dropout : float
        Dropout probability for the decoder.

    attention : Attention
        Attention module used by the decoder.

    is_bi : bool
        Flag indicating if the encoder is bidirectional.

    Attributes
    ----------
    dropout : float
        Dropout probability for the decoder.

    attention : Attention
        Attention module used by the decoder.

    output_size : int
        Size of the target vocabulary.

    embedding : nn.Embedding
        Embedding layer.

    cell : nn.Module
        Decoder's LSTM/GRU/RNN cell.

    out : nn.Linear
        Linear layer to transform the decoder's output.

    Methods
    -------
    forward(trg, encoder_outputs, hidden, cell=None)
        Forward pass of the decoder.

    """
    def __init__(self, output_size, embed_size, enc_hid_dim, dec_hid_dim, num_layers, cell_mode, dropout, attention, is_bi):
        super().__init__()

        self.dropout = dropout
        self.attention = attention
        self.output_size = output_size

        self.embedding = nn.Embedding(output_size, embed_size, padding_idx=hindi_alphabet_to_index['.'])
        cell = cell_type(cell_mode)
        self.cell_mode = cell_mode.lower()

        if is_bi:
            self.cell = cell((enc_hid_dim * 2) + embed_size, dec_hid_dim, num_layers, dropout=dropout, bidirectional=False, batch_first=False)
        else:
            self.cell = cell(enc_hid_dim + embed_size, dec_hid_dim, num_layers, dropout=dropout, bidirectional=False, batch_first=False)

        self.out = nn.Linear(dec_hid_dim, output_size)

    def forward(self, trg, encoder_outputs, hidden, cell=None):
        """
        Forward pass of the decoder.

        Parameters
        ----------
        trg : torch.Tensor
            Target tensor for a single time step. 

        encoder_outputs : torch.Tensor
            Outputs of the encoder. 

        hidden : torch.Tensor
            Hidden state of the decoder.

        cell : torch.Tensor, optional
            Cell state of the decoder for LSTM. 

        Returns
        -------
        prediction : torch.Tensor
            Output prediction for a single time step. 

        hidden : torch.Tensor
            Hidden state of the decoder. 

        cell : torch.Tensor, optional
            Cell state of the decoder for LSTM. 

        """
        attention = self.attention(encoder_outputs, hidden).unsqueeze(1)
        context = torch.bmm(attention, encoder_outputs).permute(1, 0, 2)

        embedded = self.embedding(trg.unsqueeze(0))
        cell_input = torch.cat((embedded, context), dim=2)

        if self.cell_mode == 'lstm':
            outputs, (hidden, cell) = self.cell(cell_input, (hidden.unsqueeze(0), cell.unsqueeze(0)))
            prediction = self.out(outputs.squeeze(0))
            return prediction, hidden.squeeze(0), cell.squeeze(0)

        outputs, hidden = self.cell(cell_input, hidden.unsqueeze(0))
        prediction = self.out(outputs.squeeze(0))
        return prediction, hidden.squeeze(0)


In [16]:
class Seq2Seq(nn.Module):
    """
    Seq2Seq model that combines an encoder and a decoder.

    Parameters
    ----------
    encoder : Encoder
        Encoder module.

    decoder : Decoder
        Decoder module.

    device : torch.device
        Device to run the model on.

    Attributes
    ----------
    encoder : Encoder
        Encoder module.

    decoder : Decoder
        Decoder module.

    device : torch.device
        Device to run the model on.

    Methods
    -------
    forward(source_batch, target_batch, teacher_forcing_ratio=0.5)
        Forward pass of the Seq2Seq model.

    """
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, source_batch, target_batch, teacher_forcing_ratio=0.5):
        """
        Forward pass of the Seq2Seq model.

        Parameters
        ----------
        source_batch : torch.Tensor
            Input source sequences. Shape [batch size, source length].

        target_batch : torch.Tensor
            Target sequences. Shape [batch size, target length].

        teacher_forcing_ratio : float, optional
            The probability of using teacher forcing during training.

        Returns
        -------
        outputs : torch.Tensor
            Decoder outputs for each time step. Shape [target length, batch size, target vocab size].

        """
        batch_size, max_len = target_batch.shape
        target_vocab_size = self.decoder.output_size

        outputs = torch.zeros(max_len, batch_size, target_vocab_size).to(self.device)

        if self.encoder.cell_mode == 'lstm':
            encoder_outputs, hidden, cell = self.encoder(source_batch)

            trg = target_batch[:, 0]
            for i in range(1, max_len):
                prediction, hidden, cell = self.decoder(trg, encoder_outputs, hidden, cell)
                outputs[i] = prediction

                if np.random.random() < teacher_forcing_ratio:
                    trg = target_batch[:, i]
                else:
                    trg = prediction.argmax(1)

            return outputs

        else:
            encoder_outputs, hidden = self.encoder(source_batch)

            trg = target_batch[:, 0]
            for i in range(1, max_len):
                prediction, hidden = self.decoder(trg, encoder_outputs, hidden)
                outputs[i] = prediction

                if np.random.random() < teacher_forcing_ratio:
                    trg = target_batch[:, i]
                else:
                    trg = prediction.argmax(1)

            return outputs


In [17]:
def accuracy_calc(target_seq,seq2,mode='full',device=device):# predicted
    eos_index=(target_seq==hindi_alphabet_to_index['>']).nonzero()
    eos_idx=eos_index[:,1]
    
    correct=torch.Tensor([0]).to(device)
    correct_chars=torch.Tensor([0]).to(device)
    tot_chars=torch.Tensor([0]).to(device)
    for iterate,idx in enumerate(eos_idx):
        inputter=seq2[iterate][:idx]
        outputter=target_seq[iterate][:idx]
        if torch.all(torch.eq(inputter,outputter)):
            correct+=1
            correct_chars+=idx
            tot_chars+=idx
        else:
            correct_chars+=torch.sum(inputter == outputter).item()
            tot_chars+=idx
            
#         print(correct,correct_chars,tot_chars)
        
    return correct.item(),correct_chars.item(),tot_chars.item()
            
            
        
    

In [18]:
def train(seq2seq, iterator, optimizer, criterion):
    
    
    seq2seq.train()
    
    epoch_loss = 0
    correct=0
    correct_char=0
    tot_char=0
    
    relax_acc=0
    
    
    for batch in iterator:
        optimizer.zero_grad()
        outputs = seq2seq(batch['input'], batch['output'])
        batch_label=batch['output'].transpose(0,1)
        batch_size=len(batch['output'])
        
        _, predicted = torch.max(outputs, dim=2)
        outputs_flatten = outputs.view(-1, outputs.shape[-1])
        trg_flatten = batch_label.reshape(-1)
        

        trg_flatten.requires_grad=False
        loss = criterion(outputs_flatten, trg_flatten)
        correct_temp,correct_chars_temp,tot_chars_temp=accuracy_calc(batch['output'],predicted.transpose(0,1))
        
        #___________
        
        correct+=correct_temp
        correct_char+=correct_chars_temp
        tot_char+=tot_chars_temp
        
        
        #_______________
        

        loss.backward()
        optimizer.step()
        

        epoch_loss += loss.item()
        


    return epoch_loss / len(iterator), correct/(len(iterator)*16),correct_char/tot_char

In [19]:
def evaluate(seq2seq, iterator, criterion):
    seq2seq.eval()

    epoch_loss = 0
    correct=0
    correct_char=0
    tot_char=0
    
    relax_acc=0
    
    with torch.no_grad():
        for batch in iterator:
            outputs = seq2seq(batch['input'], batch['output'],teacher_forcing_ratio=0)
            batch_label=batch['output'].transpose(0,1)
            batch_size=len(batch['output'])


            _, predicted = torch.max(outputs, dim=2)
            #print('wow_preds',predicted.shape)

            outputs_flatten = outputs.view(-1, outputs.shape[-1])
            trg_flatten = batch_label.reshape(-1)

            loss = criterion(outputs_flatten, trg_flatten)
            
            correct_temp,correct_chars_temp,tot_chars_temp=accuracy_calc(batch['output'],predicted.transpose(0,1))
        
            #___________

            correct+=correct_temp
            correct_char+=correct_chars_temp
            tot_char+=tot_chars_temp

            #_______________       
            
            epoch_loss += loss.item()
            

    return epoch_loss / len(iterator), correct/(len(iterator)*16),correct_char/tot_char



In [20]:
def epoch_time(start_time, end_time):
    e_time = end_time - start_time
    mins = e_time // 60
    secs = e_time%60
    return mins, secs,

In [21]:
def count_params(model):
    return sum(param.numel() for param in model.parameters() if param.requires_grad)

In [22]:
import time
import random

In [23]:
import torch.optim as optim
def make_model(train_iterator,valid_iterator, enc_embed_size, enc_hid_size, dec_embed_size, dec_hid_size,\
               num_layers, cell_mode,\
                 dropout, is_bi, epochs=20):
    
    
    E = Encoder(30,  embed_size=enc_embed_size, enc_hid_size=enc_hid_size, dec_hid_size=dec_hid_size,\
                num_layers=num_layers, cell_mode=cell_mode, dropout=dropout, is_bi=is_bi).to(device)
    
    
    A = Attention(enc_hid_dim=enc_hid_size, dec_hid_dim=dec_hid_size, is_bi=is_bi).to(device)

    D = Decoder(68, embed_size=dec_embed_size, enc_hid_dim=enc_hid_size, dec_hid_dim=dec_hid_size,\
                num_layers=1, cell_mode=cell_mode,\
                 dropout=dropout, attention=A, is_bi=is_bi).to(device)

    S=Seq2Seq(E,D,device)
    S.to(device)    
    print(f'The model has {count_params(S):,} trainable parameters')
    
    optimizer = optim.Adam(S.parameters())
    criterion = nn.CrossEntropyLoss(ignore_index=hindi_alphabet_to_index['.'])
    criterion=criterion.to(device)
    
    best_valid_loss = float('inf')
#     return S

    for epoch in range(epochs):    
        start_time = time.time()
        train_loss,train_acc,train_stuff = train(S, train_iterator, optimizer, criterion)
        valid_loss,valid_acc,val_stuff = evaluate(S, valid_iterator, criterion)
        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(S.state_dict(), 'model1.pt')

        # it's easier to see a change in perplexity between epoch as it's an exponential
        # of the loss, hence the scale of the measure is much bigger
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs:.2f}s')
        print(f'\t Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
        print(f'\t Relaxed Train. Acc: {train_stuff*100:.2f}% | Relaxed Val. Acc: {val_stuff*100:.2f}%')
        
        
    return S

    

# Make custom model 
pass your data loaders, configurations etc.

In [24]:
train_iterator=train_val_dataloader
valid_iterator=test_dataloader
SS=make_model(train_iterator,valid_iterator,cell_mode='lstm',dec_embed_size=16,dec_hid_size=128,\
              dropout=0.2,enc_embed_size=32,enc_hid_size=256,epochs=30,is_bi=True,num_layers=3)

In [29]:
#Uncomment to Save Model

# torch.save(SS, 'attn_model.model')



In [30]:
# # Uncomment to laod model:

# the_model = torch.load('attn_model.model')
the_model=SS

In [28]:
test_full_dataloader = DataLoader(test_data, batch_size=len(test_data),shuffle=False)

In [29]:
#run on test data

the_model.eval()
preds=the_model(next(iter(test_full_dataloader))['input'],next(iter(test_full_dataloader))['output'],teacher_forcing_ratio=0)

In [30]:
_, predicted = torch.max(preds, dim=2)

In [34]:
def word_from_batch(batch):
    wordlet=[]
    for i in range(len(batch)):
        wordlet.append(word_from_torchies(batch[i],index_to_hindi_alphabet))
    return wordlet
        

In [35]:
def word_from_batch_eng(batch):
    wordlet=[]
    for i in range(len(batch)):
        wordlet.append(word_from_torchies(batch[i],index_to_english_alphabet))
    return wordlet

# Utils To view the words

Pass predictions.transpose(0,1)

Pass batch['output']

In [35]:
import math

In [36]:
def word_from_torchies(torchie1,index_toalp):
    torchie=torchie1.cpu().numpy()
    return word_from_vecs(torchie,index_toalp,False)

In [37]:
def word_from_batch(batch):
    wordlet=[]
    for i in range(len(batch)):
        wordlet.append(word_from_torchies(batch[i],index_to_hindi_alphabet))
    return wordlet
        

In [38]:
def word_from_batch_eng(batch):
    wordlet=[]
    for i in range(len(batch)):
        wordlet.append(word_from_torchies(batch[i],index_to_english_alphabet))
    return wordlet

In [39]:
test_preds=word_from_batch(predicted.transpose(0,1))

In [41]:
test_actual=word_from_batch(next(iter(test_full_dataloader))['output'])

In [42]:
test_input=word_from_batch_eng(next(iter(test_full_dataloader))['input'])

In [36]:
test_preds=word_from_batch(predicted.transpose(0,1))

In [185]:
import pandas as pd
datas={'Ground truth':test_actual,'Predictions Attention':test_preds, "Predictions Vanilla":va}
df=pd.DataFrame(data=datas,index=test_input)


In [191]:
xx=df[df['Predictions Attention']!=df['Predictions Vanilla']]

In [194]:
yy=xx[xx['Predictions Attention']==xx['Ground truth']]


In [195]:
yy

,Ground truth,Predictions Attention,Predictions Vanilla
shikayatkarta,शिकायतकर्ता,शिकायतकर्ता,शिकटयाकर्ता
kaarniyaan,कार्नियां,कार्नियां,कारनियाँ
holt,होल्ट,होल्ट,हॉल्ट
laigikata,लैंगिकता,लैंगिकता,लैईगीकता
vankshetra,वनक्षेत्र,वनक्षेत्र,वंक्षेत्र
...,...,...,...
francisco,फ्रांसिस्को,फ्रांसिस्को,फ्रैंसिस्को
raghavan,राघवन,राघवन,रघवन
thaki,थकी,थकी,ठकी
punchang,पंचांग,पंचांग,पुंचांग


In [197]:
zz=xx[xx['Predictions Vanilla']==xx['Ground truth']]

In [198]:
zz

,Ground truth,Predictions Attention,Predictions Vanilla
twitters,ट्विटर्स,ट्वियरटर्स,ट्विटर्स
ukhrul,उखरुल,उखरूल,उखरुल
iqbal,इक़बाल,इकबाल,इक़बाल
umanath,उमानाथ,उमनाथ,उमानाथ
sushrushaa,सुश्रुषा,सुष्रशषा,सुश्रुषा
...,...,...,...
prapt,प्राप्त,प्रप्त,प्राप्त
kanthahar,कंठहार,कंठहर,कंठहार
oxide,ऑक्साइड,आक्साइड,ऑक्साइड
shbana,शबाना,श्बाना,शबाना


In [42]:
df_correct

,Ground truth,Predictions
sikhaaega,सिखाएगा,सिखाएगा
tirunelveli,तिरुनेलवेली,तिरुनेलवेली
independence,इंडिपेंडेंस,इंडिपेंडेंस
speshiyon,स्पेशियों,स्पेशियों
kolhapur,कोल्हापुर,कोल्हापुर
...,...,...
khairati,खैराती,खैराती
deshke,देशके,देशके
seho,सेहो,सेहो
belcha,बेलचा,बेलचा


In [43]:
df_incorrect=df[df['Ground truth']!=df['Predictions']]

In [44]:
df_incorrect

,Ground truth,Predictions
learn,लर्न,लियन
twitters,ट्विटर्स,ट्वियरटर्स
shurooh,शुरूः,शुरूह
ajhar,अजहर,अझर
karaar,क़रार,करार
...,...,...
miti,मिति,मिटी
saflata,सफ़लता,सफलाता
shbana,शबाना,श्बाना
khaatootolaa,खातूटोला,खातूतोला
